In [1]:
# importing the libraries
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import joblib
import numpy as np
import torch.nn.functional as F  
import os

In [ ]:
# Loading the Pretrained Model
model_path = "C:/Users/aadit/OneDrive/Desktop/IndiaAI_CyberGuard Hackathon/IndiaAIHackathon/distilbert-fine-tuned"  
model = AutoModelForSequenceClassification.from_pretrained(model_path, local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)  
label_encoder = joblib.load(f'{model_path}/label_encoder.pkl')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

c:\Users\aadit\anaconda\envs\pytorch_env\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [3]:
# Predicting the Model with Test data
def predict_with_confidence(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model(**inputs)
    
    probabilities = F.softmax(outputs.logits, dim=-1)

    predicted_index = torch.argmax(probabilities, dim=-1).item()
    
    confidence_score = probabilities[0, predicted_index].item()
    
    predicted_label = label_encoder.inverse_transform([predicted_index])[0]
    
    return predicted_label, confidence_score

In [4]:
# Example usage
text = "Sensitive financial data was leaked."
predicted_label, confidence_score = predict_with_confidence(text)
print(f"Predicted Label: {predicted_label}, Confidence Score: {confidence_score:.4f}")

Predicted Label: Online Financial Fraud, Confidence Score: 0.7503
